In [1]:
# Multiple Participants -- Approach 2: Model Template
# Train & Test on Distinct Groups of Participants
# Approach 2: Use Small % of Test Participant's Data (Chronological Split)

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

In [3]:
# Key Features: op_num_people, of_pose_distance, of_gaze_distance, of_confidence, ros_mistakes_session, ros_ts_robot_talked, timestamp
use_key_features = True

# Leave One Out Run: test on each participant & train on remaining
leave_one_out_run = True

# Number of Estimators for Bagging
num_estimators = 10

# Sample Percentage for Bagging
sample_percentage = 0.75

# Percentage of Test Participant Data to Use
test_percentage_to_use = 0.3

# File Name for Model Results
output_file = 'multi2_kf_knn_results.csv'

# 1 -- Load Data

In [4]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [5]:
data = [data5, data7, data9, data11, data12, data17, data18]
part = [5, 7, 9, 11, 12, 17, 18]

In [6]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

for i, d in enumerate(data):
    data[i] = d.drop(columns=remove)

In [7]:
# Remove rows where engagment NaN
for i,d in enumerate(data):
    data[i] = d[np.isfinite(d['engagement'])]
    
# Remove rows where engagment is -1
for i,d in enumerate(data):
    data[i] = d[d['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [8]:
# Main Columns
basic_cols = []
for i in data[0].columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [9]:
# Open Face Columns

of_cols = []
for i in data[0].columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [10]:
# Audio Columns

a_cols = []
for i in data[0].columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [11]:
# ROS Columns

ros_cols = []
for i in data[0].columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [12]:
# For Window Only:
non_window_features = []
window_features = []
for i in data[0].columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [13]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [14]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [15]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

for i,d in enumerate(data):
    data[i] = d[features_to_keep]

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [16]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [17]:
# Helper Function: Split a Participant's Data Chronologically 

def part_split(split, split_size):
    split = split.sort_values(['session_num', 'timestamp'], ascending=[True, True])
    bogus = split['engagement']

    split_train, split_test, bogus1, bogus2 = train_test_split(split, bogus, test_size=split_size, shuffle=False)
    return split_test, split_train

In [18]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing & Shuffle 

# train_part: an array of participants 
# test_part: an array of participants
# split_size: how much of each test participant's data to use for validation
def split(train_part, test_part, split_size):
    train_data = []
    for p in train_part:
        i = part.index(p)
        train_data.append(data[i].copy())
    
    val_data = []
    test_data = []
    for p in test_part:
        i = part.index(p)
        if split_size > 0:
            p_test, p_train = part_split(data[i].copy(), split_size)
            val_data.append(p_train)
            test_data.append(p_test)
        else:
            test_data.append(data[i].copy())
        
    train_data = pd.concat(train_data, ignore_index=True, sort=True)
    val_data = pd.concat(val_data, ignore_index=True, sort=True)
    test_data = pd.concat(test_data, ignore_index=True, sort=True)
    
    # Sort Training & Validation Data 
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    train_data = train_data.sample(frac=1)
    val_data = val_data.sample(frac=1)
    val_data = val_data.sample(frac=1)
    val_data = val_data.sample(frac=1)

    X_train = train_data.drop(columns=['engagement', 'session_num', 'participant'])        
    X_val = val_data.drop(columns=['engagement', 'session_num', 'participant'])  
    X_test = test_data.drop(columns=['engagement', 'session_num', 'participant'])        

    y_train = train_data['engagement']
    y_val = val_data['engagement']
    y_test = test_data['engagement']

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_val[c] = (X_val[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_val[c] = (X_val[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_val[c] = X_val[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_val[c] = X_val[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_val, y_val, X_test, y_test

In [19]:
# MODEL HERE
# Multiple Participants Approach 2: 
# Approach 2: Use Small % of Test Participant's Data (Chronological Split)

# Inputs: X_train, y_train, X_val, y_val, X_test, y_test

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.ensemble import BaggingClassifier
from sklearn import neighbors

def model(X_train, y_train, X_val, y_val, X_test, y_test):
    # Model Here: must create pred & scores arrays
    #clf = BaggingClassifier(n_estimators=num_estimators, max_samples=sample_percentage)
    
    X_train_new = X_train.append(X_val)
    y_train_new = y_train.append(y_val)
    
    idx = np.random.permutation(len(X_train_new))
    X_train_new = X_train_new.iloc[idx]
    y_train_new = y_train_new.iloc[idx]
    X_train_new = X_train_new.reset_index(drop=True)
    y_train_new = y_train_new.reset_index(drop=True)

    clf = neighbors.KNeighborsClassifier()
    clf.fit(X_train_new, y_train_new)
    scores = clf.predict_proba(X_test.values)[:,1]
    pred = [round(value) for value in scores]   
        
    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    try:
        auc = roc_auc_score(y_test, scores)
    except:
        auc = np.nan
    results['auc'] = auc

    precision = metrics.precision_score(y_test, pred, average=None)
    recall = metrics.recall_score(y_test, pred, average=None)
    f1 = metrics.f1_score(y_test, pred, average=None)

    results['precision_0'], results['precision_1'] = precision[0], precision[1]
    results['recall_0'], results['recall_1'] = recall[0], recall[1]
    results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [20]:
all_results = []
for n in range(1, len(part)):    
    if leave_one_out_run:
        if n != len(part)-1:
            continue
    
    comb = combinations(part, n)
    print()
    
    for c in comb:
        print()
        new_results = {}
        
        # Determine Train - Test Participants
        train_part = [x for x in c]
        test_part = [p for p in part if p not in c]
        print('Train:', train_part, '\tTest:', test_part)

        # Train - Test Split (and preprocessing)
        X_train, y_train, X_val, y_val, X_test, y_test = split(train_part, test_part, test_percentage_to_use)
        #X_train = X_train.reset_index(drop=True)
        #y_train = y_train.reset_index(drop=True)
        #X_val = X_val.reset_index(drop=True)
        #y_val = y_val.reset_index(drop=True)

        # Evaluate on Test  
        results = model(X_train, y_train, X_val, y_val, X_test, y_test)
        print(results)
        results['train_participants'] = ", ".join(str(x) for x in train_part)
        results['test_participants'] = ", ".join(str(x) for x in test_part)
        all_results.append(results)



Train: [5, 7, 9, 11, 12, 17] 	Test: [18]
{'precision_0': 0.4083107497741644, 'auc': 0.6430813684336459, 'accuracy': 0.7025159461374911, 'recall_1': 0.8428502879078695, 'precision_1': 0.7743001983689662, 'recall_0': 0.3062330623306233, 'f1_0': 0.3499806426635695, 'f1_1': 0.8071223434807582}

Train: [5, 7, 9, 11, 12, 18] 	Test: [17]
{'precision_0': 0.7489711934156379, 'auc': 0.7291507160656097, 'accuracy': 0.6744246861924686, 'recall_1': 0.8454913880445796, 'precision_1': 0.6397087006515907, 'recall_0': 0.4918918918918919, 'f1_0': 0.5938009787928222, 'f1_1': 0.7283438795548768}

Train: [5, 7, 9, 11, 17, 18] 	Test: [12]
{'precision_0': 0.8149113660062565, 'auc': 0.8137749875931608, 'accuracy': 0.7652211066011536, 'recall_1': 0.850463352990733, 'precision_1': 0.7307274701411509, 'recall_0': 0.6775032509752926, 'f1_0': 0.7398816568047337, 'f1_1': 0.7860619038349231}

Train: [5, 7, 9, 12, 17, 18] 	Test: [11]
{'precision_0': 0.39844851904090267, 'auc': 0.608542695681123, 'accuracy': 0.68086

In [21]:
results = pd.DataFrame(columns=['train_participants', 'test_participants', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1'])
results = results.append(all_results, ignore_index=True, sort=True)

In [22]:
results.head()

,accuracy,auc,f1_0,f1_1,precision_0,precision_1,recall_0,recall_1,test_participants,train_participants
0,0.702516,0.643081,0.349981,0.807122,0.408311,0.774300,0.306233,0.842850,18,"5, 7, 9, 11, 12, 17"
1,0.674425,0.729151,0.593801,0.728344,0.748971,0.639709,0.491892,0.845491,17,"5, 7, 9, 11, 12, 18"
2,0.765221,0.813775,0.739882,0.786062,0.814911,0.730727,0.677503,0.850463,12,"5, 7, 9, 11, 17, 18"
3,0.680863,0.608543,0.339034,0.789650,0.398449,0.753874,0.295039,0.828990,11,"5, 7, 9, 12, 17, 18"
4,0.641364,0.696993,0.665838,0.613022,0.810356,0.508088,0.565064,0.772581,9,"5, 7, 11, 12, 17, 18"


In [23]:
results.to_csv(output_file, index=False)